In [1]:
%cd ..

/home/alta/Conversational/OET/al826/2022/shortcuts/data_pruning


In [12]:
from src.system_loader import EnsembleLoader, SystemLoader
from src.utils.evaluation import get_seed_accuracy, get_accuracy

system = EnsembleLoader('trained_models/hpc/baseline') 
seed_preds  = system.load_seed_preds('paws', mode='test')
labels = SystemLoader.load_labels('paws', mode='test')
print(get_seed_accuracy(seed_preds, labels))

Reusing dataset paws (/home/alta/Conversational/OET/al826/.cache/huggingface/datasets/paws/labeled_final/1.1.0/8d567c6472623f42bd2cc635cad06932d0f0cd2f897db56013c1180f4317d338)


  0%|          | 0/3 [00:00<?, ?it/s]

[91.3375, 91.57499999999999, 91.4875, 91.9625, 91.2]
(91.51249999999999, 0.2589642832515712)


In [13]:
import numpy as np

sample_ids = [k for k in labels]
seed_1 = np.array([seed_preds[0][k] for k in sample_ids])
seed_2 = np.array([seed_preds[1][k] for k in sample_ids])
seed_3 = np.array([seed_preds[2][k] for k in sample_ids])
seed_4 = np.array([seed_preds[3][k] for k in sample_ids])
seed_5 = np.array([seed_preds[4][k] for k in sample_ids])
labels_np = np.array([labels[k]     for k in sample_ids])

In [14]:
print(len(labels_np))

8000


In [15]:
incorrect_1 = [k for k in sample_ids if seed_preds[0][k] != labels[k]]
incorrect_2 = [k for k in sample_ids if seed_preds[1][k] != labels[k]]

In [16]:
overlap = len(set(incorrect_1) & set(incorrect_2))
union   = len(set(incorrect_1) | set(incorrect_2))

print(overlap/union)

0.5463800904977375


In [17]:
overlap = (seed_4 == labels_np) == (seed_1 == labels_np)
print(overlap.sum()/labels_np.shape[0])

0.95075


In [ ]:
from src.